<img src="https://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png" width="600px">

## Mapping DYNAMICO output

- Author: Patrick Brockmann
- Version: 25/11/2022

### Testing pyvista and different backends

In [1]:
import numpy as np
import xarray as xr
import pyvista as pv
import matplotlib

In [2]:
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
ds = xr.open_dataset("https://thredds-su.ipsl.fr/thredds/dodsC/ipsl_thredds/brocksce/ICO/ICO.79.1jour.native.1_19790101_19790101_1D_inca_ges.nc")
ds

<xarray.Dataset>
Dimensions:               (cell: 16002, nvertex: 6, presnivs: 79,
                           time_counter: 24, axis_nbounds: 2)
Coordinates:
    lat                   (cell) float32 ...
    lon                   (cell) float32 ...
  * presnivs              (presnivs) float32 1.012e+05 1.009e+05 ... 5.297 1.483
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 1979-01-01T00:30:00 ....
    time_instant          (time_counter) datetime64[ns] ...
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables: (12/29)
    bounds_lon            (cell, nvertex) float32 ...
    bounds_lat            (cell, nvertex) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    time_instant_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    pmid                  (time_counter, presnivs, cell) float64 ...
    ...                    ...
    vmrmcf                (time_counter, presnivs, cell) float64 ...
    vmrbe7                (time_counter, presnivs, cell) float64 ...
    vmrn2o                (time_counter, presnivs, cell) float64 ...
    vmrch4                (time_counter, presnivs, cell) float64 ...
    vmrco                 (time_counter, presnivs, cell) float64 ...
    vmrco2bih             (time_counter, presnivs, cell) float64 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.0|hdf5lib...
    name:                            inca1d_ges
    description:                     Created by xios
    title:                           Created by xios
    Conventions:                     CF-1.6
    timeStamp:                       2022-Apr-05 08:59:15 GMT
    uuid:                            8982bdb7-b650-45c4-896c-e228bec0cb5a
    history:                         Fri Apr 15 12:05:37 2022: ncatted -O -a ...
    NCO:                             4.6.9
    DODS_EXTRA.Unlimited_Dimension:  time_counter

In [4]:
blon = ds['bounds_lon'].to_numpy()
blat = ds['bounds_lat'].to_numpy()
nvertex = blon.shape[-1]
print(nvertex)

blon = blon.reshape(-1, nvertex)
blat = blat.reshape(-1, nvertex)
print(blon.shape)

arr = ds['bounds_lon'].to_numpy()
blon = arr.reshape(-1, arr.shape[-1])
arr = ds['bounds_lat'].to_numpy()
blat = arr.reshape(-1, arr.shape[-1])

6
(16002, 6)


In [5]:
R = 0.999
deg2rad = np.pi/180.
x = R * np.cos(blat*deg2rad)*np.cos(blon*deg2rad)
y = R * np.cos(blat*deg2rad)*np.sin(blon*deg2rad)
z = R * np.sin(blat*deg2rad)

In [6]:
points = np.stack((x,y,z), axis=2).reshape(x.size, 3)
faces = np.arange(x.shape[0] * nvertex).reshape(x.shape[0], nvertex)
faces = np.insert(faces, 0, nvertex, axis=1)

In [7]:
mesh = pv.PolyData(points, faces)
mesh

PolyData,Information
N Cells,16002
N Points,96012
N Strips,0
X Bounds,"-9.990e-01, 9.990e-01"
Y Bounds,"-9.988e-01, 9.988e-01"
Z Bounds,"-9.989e-01, 9.989e-01"
N Arrays,0


In [8]:
mesh = pv.PolyData(points, faces)
mesh.clean(inplace=True, tolerance=0.001)
mesh

PolyData,Information
N Cells,16002
N Points,32000
N Strips,0
X Bounds,"-9.990e-01, 9.990e-01"
Y Bounds,"-9.988e-01, 9.988e-01"
Z Bounds,"-9.989e-01, 9.989e-01"
N Arrays,0


In [9]:
var = ds['temp']
print(var.shape)

(24, 79, 16002)


In [10]:
kmax = var.shape[1]-1
lmax = var.shape[0]-1
kmax, lmax

(78, 23)

In [11]:
var = ds['temp'][0,0]
varmin = np.min(var)
varmax = np.max(var)
print("%f %f" %(varmin, varmax))

233.364045 302.943871


In [12]:
! wget -nc https://thredds-su.ipsl.fr/thredds/fileServer/ipsl_thredds/brocksce/pyvista/continents.vtk

Fichier «continents.vtk» déjà présent ; pas de récupération.



In [13]:
import vtk
vtk.vtkLogger.SetStderrVerbosity(vtk.vtkLogger.VERBOSITY_OFF)

### With pythreejs backend

 * The position has to be saved.
 * No scalar bar implemented yet

In [14]:
plotter = pv.Plotter()
plotter.set_background('lightgray')

continents = pv.PolyData('continents.vtk')
cont_actor = plotter.add_mesh(continents, color='white')
mesh_actor = plotter.add_mesh(mesh, scalars=var, clim=[240, 290], cmap='plasma', show_edges=True)
viewer = plotter.show(jupyter_backend='pythreejs', return_viewer=True)

def plot(show_edges=True, kindex=0, lindex=0, clim=[240, 290], cmap='plasma'):
    global viewer, mesh_actor
    save = viewer.camera.position
    var = ds['temp'][lindex,kindex]
    plotter.remove_actor(mesh_actor)
    cont_actor = plotter.add_mesh(continents, color='white')
    mesh_actor = plotter.add_mesh(mesh, scalars=var, clim=clim, cmap=cmap, show_edges=show_edges)
    plotter.update_scalars(var, mesh=mesh)
    viewer = plotter.show(jupyter_backend='pythreejs', return_viewer=True)
    viewer.camera.position = save
    viewer.camera.lookAt((0,0,0))
    display(viewer)

a = interact(plot,
        cmap = matplotlib.colormaps(),
        show_edges = True, 
        kindex = (0,kmax,1),
        lindex = (0,lmax,1),
        clim = widgets.FloatRangeSlider(min=200, max=400, step=10, value=[240, 290])
)

interactive(children=(Checkbox(value=True, description='show_edges'), IntSlider(value=0, description='kindex',…

### With panel backend

 * There a blink each time a slider value is changed.

In [16]:
plotter = pv.Plotter()
plotter.set_background('lightgray')

continents = pv.PolyData('continents.vtk')
cont_actor = plotter.add_mesh(continents, color='white')
mesh_actor = plotter.add_mesh(mesh, scalars=var, clim=[240, 290], cmap='plasma', show_edges=True)
viewer = plotter.show(jupyter_backend='panel', return_viewer=True)

def plot(show_edges=True, kindex=0, lindex=0, clim=[240, 290], cmap='plasma'):
    global plotter, mesh_actor, viewer
    var = ds['temp'][lindex,kindex]
    plotter.remove_actor(mesh_actor, render=False, reset_camera=False)
    mesh_actor = plotter.add_mesh(mesh, scalars=var, clim=clim, cmap=cmap, show_edges=show_edges)
    plotter.update_scalars(var, mesh=mesh)
    plotter.show(jupyter_backend='panel', return_viewer=True)
    display(viewer)

a = interact(plot,
        cmap = matplotlib.colormaps(),
        show_edges = True, 
        kindex = (0,kmax,1),
        lindex = (0,lmax,1),
        clim = widgets.FloatRangeSlider(min=200, max=400, step=10, value=[240, 290])
)

interactive(children=(Checkbox(value=True, description='show_edges'), IntSlider(value=0, description='kindex',…

### Simplest test with pythreejs backend

 * Is there simpler than remove actor each time a widget is triggered ?

In [17]:
import vtk
vtk.vtkLogger.SetStderrVerbosity(vtk.vtkLogger.VERBOSITY_OFF)

import pyvista as pv
from ipywidgets import interact

pl = pv.Plotter()
actor = pl.add_mesh(pv.Sphere(), show_edges=True)
viewer = pl.show(jupyter_backend='pythreejs', return_viewer=True)

def plot(show_edges):
    global actor, viewer
    pl.remove_actor(actor)
    actor = pl.add_mesh(pv.Sphere(), show_edges=show_edges)
    viewer = pl.show(jupyter_backend='pythreejs', return_viewer=True)
    display(viewer)
    
a = interact(plot, show_edges=True)

interactive(children=(Checkbox(value=True, description='show_edges'), Output()), _dom_classes=('widget-interac…